# .xml (cei) to csv [charters for now]
### info

✔ means tested and working

👑 means most worthy and purposeful script in given context

### still to do:

- check dependencies for individual code parts
- consider that atomID should be saved regardless of whether there is an image
- use timeit to check performance (barely of importance now); https://lxml.de/performance.html

# import xml and set tree+root (using etree, is fastest)

In [ ]:
import xml.etree.ElementTree as ET 
tree = ET.parse('0632d68f-f907-4113-886d-d7afd25339ef.cei.xml')
root = tree.getroot()

# get all xml tags used in 1 file ✔

In [ ]:
from pprint import pprint

elemList = []

for elem in tree.iter():
    elemList.append(elem.tag)

elemList = list(set(elemList))
pprint(elemList)

# get ALL graphic urls from 1 file ✔

In [ ]:
import xml.etree.ElementTree as ET

In [ ]:
namespaces = {'atom': 'http://www.w3.org/2005/Atom', 'cei': 'http://www.monasterium.net/NS/cei'}

In [ ]:
tree = ET.parse('0632d68f-f907-4113-886d-d7afd25339ef.cei.xml')
root = tree.getroot()

In [ ]:
for tag in root.findall('.//cei:graphic', namespaces):
    attribute = tag.attrib
    print(attribute)

# get atomID and graphicURL into pandas df from 1 file ✔

In [ ]:
import xml.etree.ElementTree as ET
import pandas as pd

In [ ]:
namespaces = {'atom': 'http://www.w3.org/2005/Atom', 'cei': 'http://www.monasterium.net/NS/cei'}

In [ ]:
tree = ET.parse('0632d68f-f907-4113-886d-d7afd25339ef.cei.xml')
root = tree.getroot()

In [ ]:
atomIDs = []
attributes = []

for tag in root.findall('.//cei:graphic', namespaces):
    atomID = root[0].text
    atomIDs.append(atomID)
    attribute = tag.attrib['url'] 
    attributes.append(attribute)

In [ ]:
img_list = list(zip(atomIDs, attributes))

In [ ]:
df = pd.DataFrame(img_list).rename(columns={0: 'atom', 1: 'url'})
df

# get atomID and graphicURL into pandas df from 1 collection using os (without subdirs) ✔

In [ ]:
import os
import xml.etree.ElementTree as ET
import pandas as pd

In [ ]:
namespaces = {'atom': 'http://www.w3.org/2005/Atom', 'cei': 'http://www.monasterium.net/NS/cei'}
directoryPath = '../../data/monasterium/MOM-data/db/mom-data/metadata.charter.public/DE-StaAWuerzburg/Buergerspital'
extension = ('.cei.xml')

In [ ]:
atomIDs = []
attributes = []

with os.scandir(directoryPath) as it:
    for entry in it:
        if entry.name.endswith(extension) and entry.is_file():
            tree = ET.parse(entry.path)
            root = tree.getroot()
            for tag in root.findall('.//cei:graphic', namespaces):
                atomID = root[0].text
                atomIDs.append(atomID)
                attribute = tag.attrib['url'] 
                attributes.append(attribute)

In [ ]:
img_list = list(zip(atomIDs, attributes))
df = pd.DataFrame(img_list).rename(columns={0: 'atomID', 1: 'url'})
df

In [ ]:
import pathlib
path = pathlib.PurePath(directoryPath)
pathname = path.name

In [ ]:
df.to_csv(f'{pathname}.csv', index=False)

# 👑 get atomID and graphicURL into pandas df from collection(s) using rglob (pathlib; works for subdirectories) ✔
fastest method tested (still takes several minutes for whole batch)

In [ ]:
from pathlib import Path
import xml.etree.ElementTree as ET
import pandas as pd

In [ ]:
namespaces = {'atom': 'http://www.w3.org/2005/Atom', 'cei': 'http://www.monasterium.net/NS/cei'}
directoryPath = '\\\?/'+'C://Users/atzenhof/playground/data/monasterium/MOM-data/db/mom-data/metadata.charter.public/'
fileExtension = ('*.cei.xml')

In [ ]:
atomIDs = []
attributes = []

for file in Path(directoryPath).rglob(fileExtension):
    tree = ET.parse(file) # parsing does not consider utf-8 specific symbols (such as long dash); needs postcorrection
    root = tree.getroot()
    for tag in root.findall('.//cei:graphic', namespaces):
        atomID = root[0].text
        atomIDs.append(atomID)
        if 'url' in tag.attrib:
            attribute = tag.attrib['url'] 
            attributes.append(attribute)
        else:
            continue

In [ ]:
img_list = list(zip(atomIDs, attributes))
df = pd.DataFrame(img_list).rename(columns={0: 'atom', 1: 'url'})
df

In [ ]:
import pathlib
path = pathlib.PurePath(directoryPath)
pathname = path.name

In [ ]:
df.to_csv(f'{pathname}.csv', index=False)

In [ ]:
#df_links.to_csv(f'links_{pathname}.csv', index=False)

In [ ]:
#df_links = df[df['url'].str.contains("http://")]

In [ ]:
#print(df.to_string())

## redundant?
get atomID and graphicURL into pandas df from multiple collections (and subfolders) using rglob (pathlib)

In [ ]:
from pathlib import Path
import xml.etree.ElementTree as ET
import pandas as pd

In [ ]:
namespaces = {'atom': 'http://www.w3.org/2005/Atom', 'cei': 'http://www.monasterium.net/NS/cei'}
directoryPath = '../../data/monasterium/MOM-data/db/mom-data/metadata.charter.public/DE-StaAWuerzburg/Buergerspital'
fileExtension = ('*.cei.xml')

In [ ]:
atomIDs = []
attributes = []

for file in Path(directoryPath).rglob(fileExtension):
    tree = ET.parse(file)
    root = tree.getroot()
    for tag in root.findall('.//cei:graphic', namespaces):
        atomID = root[0].text
        atomIDs.append(atomID)
        if 'url' in tag.attrib:
            attribute = tag.attrib['url'] 
            attributes.append(attribute)
        else:
            continue

In [ ]:
img_list = list(zip(atomIDs, attributes))
df = pd.DataFrame(img_list).rename(columns={0: 'atom', 1: 'url'})
df

In [ ]:
import pathlib
path = pathlib.PurePath(directoryPath)
pathname = path.name

In [ ]:
df.to_csv(f'{pathname}.csv', index=False)

# transform referential img-.xml file into csv using lxml (relatively fast) ✔
requires lxml since etree does not have full xpath support (no getparent after using find)

In [ ]:
from pathlib import Path
from lxml import etree
import pandas as pd

In [ ]:
file = '\\\?/'+'C://Users/atzenhof/playground/data/monasterium/MOM-data/image_xmls/images_5.xml'

In [ ]:
tree = etree.parse(file)
root = tree.getroot()

In [ ]:
atomIDs = []
image_links = []
for img in root.findall('.//img', namespaces):
    atomID = img.getparent().attrib['id']
    atomIDs.append(atomID)
    image_link = img.attrib['src']
    image_links.append(image_link)

In [ ]:
img_list = list(zip(atomIDs, image_links))
df = pd.DataFrame(img_list).rename(columns={0: 'atom', 1: 'url'})
df

In [ ]:
df.to_csv('links.csv', index=False)

# 👑 transform referential img-.xml files (plural!) into csv using lxml (relatively fast) ✔

In [1]:
from pathlib import PurePath
from pathlib import Path
from lxml import etree
import pandas as pd

In [2]:
namespaces = {'atom': 'http://www.w3.org/2005/Atom', 'cei': 'http://www.monasterium.net/NS/cei'}
directoryPath = '\\\?/'+'C://Users/atzenhof/playground/data/monasterium/MOM-data/image_xmls' # path escape due to long path
fileExtension = ('*.xml')

In [3]:
atomIDs = []
image_links = []

for file in Path(directoryPath).rglob(fileExtension):
    tree = etree.parse(str(file)) # requres conversion to str since lxml does not vibe with windowspath
    root = tree.getroot()
    for img in root.findall('.//img', namespaces):
        atomID = img.getparent().attrib['id']
        atomIDs.append(atomID)
        image_link = img.attrib['src']
        image_links.append(image_link)

In [4]:
img_list = list(zip(atomIDs, image_links))
df = pd.DataFrame(img_list).rename(columns={0: 'atom', 1: 'url'})
df

,atom,url
0,"tag:www.monasterium.net,2011:/charter/AT-HHStA...",http://images.monasterium.net/img/AT-OeStA-HHS...
1,"tag:www.monasterium.net,2011:/charter/AT-HHStA...",http://images.monasterium.net/img/AT-OeStA-HHS...
2,"tag:www.monasterium.net,2011:/charter/AT-HHStA...",http://images.monasterium.net/img/AT-OeStA-HHS...
3,"tag:www.monasterium.net,2011:/charter/AT-HHStA...",http://images.monasterium.net/img/AT-OeStA-HHS...
4,"tag:www.monasterium.net,2011:/charter/AT-HHStA...",http://images.monasterium.net/img/AT-OeStA-HHS...
...,...,...
815787,"tag:www.monasterium.net,2011:/charter/BISANU/1...",https://images.monasterium.net/pics/858/132702...
815788,"tag:www.monasterium.net,2011:/charter/BISANU/1...",https://images.monasterium.net/pics/858/132702...
815789,"tag:www.monasterium.net,2011:/charter/BISANU/1...",https://images.monasterium.net/pics/858/132702...
815790,"tag:www.monasterium.net,2011:/charter/BISANU/1...",https://images.monasterium.net/pics/858/I.._IE...


In [ ]:
#path = pathlib.PurePath(directoryPath)
path = PurePath(directoryPath)
pathname = path.name

In [ ]:
df.to_csv(f'{pathname}.csv', index=False)

# index-based xml scrape (clunky, not recommended)
This is bad because it is rigid and depend on the .xml not changing the structure. No index may change - all .xml-tags must be placed in the same way, each time.
One possible error visible: [6] not always the case for first level after root since copied charters have to be positioned at [7], 6 is taken by atom:link

## get atom_Id ✔

In [ ]:
root[0].text

## get idno ✔

In [ ]:
(root[6][0][1][0].attrib).get('id')

## get specific img_link ✔

In [ ]:
(root[6][0][1][1][2][1][0].attrib).get('url')

## get all img_links from 1 xml ✔

In [ ]:
level_witnessOrig = root[6][0][1][1][2]
for child in level_witnessOrig:
    if child.tag == '{http://www.monasterium.net/NS/cei}figure':
        print((child[0].attrib).get('url'))
    else:
        pass